<a href="https://colab.research.google.com/github/eldercamposds/AnaliseDeSentimento/blob/SupervisionadoVSRegras/SupervisionadoVsRegras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score


from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
tweets = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NLP/7.Analise de sentimentos/Tweets2.csv")

In [ ]:
tweets.shape

(73996, 4)

In [ ]:
tweets.head()

,id,local,sentiment,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [ ]:
tweets.groupby(["sentiment"]).size()

,0
sentiment,
Irrelevant,12990
Negative,22542
Neutral,18318
Positive,20832


In [ ]:
tweets.loc[tweets.sentiment == 'Irrelevant', "sentiment"] = "Neutral"

In [ ]:
tweets.groupby(["sentiment"]).size()

,0
sentiment,
Negative,22542
Neutral,31308
Positive,20832


In [ ]:
tweets = tweets.dropna(subset=["text"])
tweets.reset_index(drop=True, inplace=True)

In [14]:
tweets.groupby(["sentiment"]).size()

,0
sentiment,
Negative,22358
Neutral,30983
Positive,20655


#Supervisionado

In [15]:
max_words=100

In [17]:
token = Tokenizer(num_words=max_words)
token.fit_on_texts(tweets["text"].values) #criando o modelo

In [18]:
X = token.texts_to_sequences(tweets["text"].values)#gerando tokens
X = pad_sequences(X, padding="post", maxlen=max_words) #ccompletar tokens com 0 a direita

In [19]:
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(tweets["sentiment"])
print(y)

[2 2 2 ... 2 2 2]


In [20]:
y  = to_categorical(y) #one hot encoder
print(y)

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)
X_test

array([[26, 39, 23, ...,  0,  0,  0],
       [ 1,  7, 88, ...,  0,  0,  0],
       [24, 57,  7, ...,  0,  0,  0],
       ...,
       [23,  5, 18, ...,  0,  0,  0],
       [ 9, 21, 51, ...,  0,  0,  0],
       [ 6, 18,  9, ...,  0,  0,  0]], dtype=int32)

In [22]:
modelo = Sequential()
modelo.add(Embedding(input_dim = len(token.word_index), output_dim=128))
modelo.add(SpatialDropout1D(0.2))#auemnta a independencia entre as caracteristicas
modelo.add(LSTM(units=196, dropout=0.2, recurrent_dropout=0, activation="tanh", recurrent_activation="sigmoid", use_bias=True))
modelo.add(Dense(units=3, activation="softmax"))

In [23]:
modelo.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
print(modelo.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d (SpatialDropout1D) │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [24]:
modelo.fit(X_train, y_train, epochs=5, batch_size=500, verbose=True)

Epoch 1/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 223s 2s/step - accuracy: 0.4189 - loss: 1.0838
Epoch 2/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 254s 2s/step - accuracy: 0.4107 - loss: 1.0855
Epoch 3/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 262s 2s/step - accuracy: 0.4191 - loss: 1.0824
Epoch 4/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 270s 2s/step - accuracy: 0.4163 - loss: 1.0838
Epoch 5/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 258s 2s/step - accuracy: 0.4185 - loss: 1.0828


In [25]:
loss, accuracy = modelo.evaluate(X_test, y_test)
print("Acurácia: ", accuracy)

925/925 ━━━━━━━━━━━━━━━━━━━━ 95s 102ms/step - accuracy: 0.4200 - loss: 1.0821
Acurácia:  0.42038583755493164
